In [1]:
from transformers import pipeline

/opt/homebrew/Caskroom/miniconda/base/envs/hermes/lib/python3.13/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
/opt/homebrew/Caskroom/miniconda/base/envs/hermes/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
classifier = pipeline(
    "zero-shot-classification",
    model="FacebookAI/roberta-large-mnli",
    hypothesis_template="Under which topic would I see the following article in the news? The topics are: {}.",
    batch_size=64
)

Some weights of the model checkpoint at FacebookAI/roberta-large-mnli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use mps:0


In [ ]:
# titles = ["OpenAI releases GPT‑5", "Global markets rally on tech earnings", "Israel appoints new defense minister", "World Cup final ends in dramatic fashion", "New AI regulations proposed by EU", "OpenAI releases GPT‑5", "Global markets rally on tech earnings", "Israel appoints new defense minister", "World Cup final ends in dramatic fashion", "New AI regulations proposed by EU", ] * 5

# topics = ["Technology", "Finance", "Sports", "Health", "Politics"]

# results = classifier(titles, candidate_labels=topics)

In [3]:
topics = ["politics", "geopolitics", "economics", "entertainment",
          "lifestyle", "sports", "science", "health", "business", "technology"]

In [15]:
# connect to the database
import pymongo
from pymongo import MongoClient
client = MongoClient("mongodb://localhost:27017/")
db = client["hermes"]
collection = db["articles"]

# loop though the articles and update the topic
total = collection.count_documents({"topic": {"$exists": False}})
print(f"Total articles: {total}")

Total articles: 51637


In [ ]:
from tqdm import tqdm

# loop through the articles and update the topic in batches of 64 articles
batch_size = 128
print(f"Processing articles in batches of {batch_size}")
for i in tqdm(range(0, total, batch_size), desc="Processing batches"):
    articles_cursor = collection.find({"topic": {"$exists": False}}).limit(batch_size)
    articles = list(articles_cursor)
    titles = [article["title"] for article in articles]

    results = classifier(titles, candidate_labels=topics)
    for article, result in zip(articles, results):
        collection.update_one({"_id": article["_id"]}, {"$set": {"topic": result["labels"][0]}})


Processing articles in batches of 128


Processing batches: 100%|██████████| 404/404 [10:41:11<00:00, 95.23s/it]   
